In [3]:
# Load pandas and numpy
import pandas as pd
import numpy as np
# For text preprocessing
import spacy
nlp = spacy.load('en_core_web_sm')

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
# Load the dataset with pandas and display the first 5 rows
filepath = '/Users/thebekhruz/Desktop/nlu/EvidenceExplorer/data/validate/dev.csv'
df = pd.read_csv(filepath)
df.head()


,Claim,Evidence,label
0,We should legalize the growing of coca leaf,"Seeing the involvement of the coca growers, th...",0
1,We should limit the use of birth control,"Although FDA-approved for contraceptive use, S...",0
2,We should prohibit flag burning,The case of Texas v. Johnson was appealed to t...,0
3,The vow of celibacy should be abandoned,Much of the encyclical is spent discussing rea...,1
4,We should further exploit natural gas,Helium is typically produced by separating it ...,0


### Check the dataset 

In [ ]:
# Check for nan values 
df.isnull().sum()


Claim       0
Evidence    0
label       0
dtype: int64

In [ ]:
# Check for duplicates
df.duplicated().sum()
    

0

In [ ]:
# Check the shape of the dataset
df.shape

(5926, 3)

### Preprocessing 

In [2]:
# Clean the text using spacy
# This involves stopword removal, lemmatization, and lowercasing and tokensization
def clean_text(text):
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop]
    return ' '.join(tokens)

# Apply the clean_text function to the text column
df['Claim_Cleaned'] = df['Claim'].apply(clean_text)
df['Evidence_Cleaned'] = df['Evidence'].apply(clean_text)



NameError: name 'df' is not defined